# CrewAI Multi-Agents Demo

# Installs

# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import configparser
import openai
import os

config = configparser.ConfigParser()
config.read('../pruebas_openai/config.ini')

OPENAI_MODEL = config['DEFAULT']['OPENAI_MODEL']
openai.api_key = config['DEFAULT']['OPENAI_API_KEY']
OPENAI_API_KEY = config['DEFAULT']['OPENAI_API_KEY']
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from langchain.chat_models import ChatOpenAI
import requests
from IPython.display import Markdown
import pandas as pd

# Artículos

In [ ]:
data = pd.read_csv("../data/contenido_2023_04_05_matriz_not_julia.csv")
data = data.iloc[[23]]

In [ ]:
# Texto de ejemplo
texto_noticia = data['contenido'].values[0]
texto_noticia

# Define Agents

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
llm_openai = True

In [ ]:
# Usando OpenAI
if llm_openai:
    llm = ChatOpenAI(model_name=OPENAI_MODEL,
                    api_key= OPENAI_API_KEY,
                    temperature=0.2)

# Usando Salamandra-2B
else:
    model_name = "BSC-LT/salamandra-2b"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    text_generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
    )

    llm = HuggingFacePipeline(pipeline=text_generator)

In [ ]:
def crear_agente(tipo, profesion, genero, politica):
    pronombre = "ella" if genero == "mujer" else "él"
    articulo = "una" if genero == "mujer" else "un"
    role = f"{tipo.capitalize()} ({profesion}, {genero}, {politica})"

    contexto = (
        f"{articulo.capitalize()} {profesion} con identidad de género {genero} "
        f"y orientación política {politica}. {pronombre.capitalize()} aporta "
        f"una perspectiva particular sobre el lenguaje y los sesgos debido a su trasfondo."
    )

    # ----------------------
    # 1. Agente Analista
    # ----------------------
    if tipo == "analista":
        goal = "Detectar estereotipos, sesgos de género o lenguaje sexista en textos periodísticos."
        backstory = contexto + " Su labor es identificar sesgos implícitos o explícitos en el texto."

    # ----------------------
    # 2. Agente Crítico
    # ----------------------
    elif tipo == "critico":
        goal = "Explicar el impacto de los sesgos detectados sobre la percepción del rol de la mujer en la sociedad."
        backstory = contexto + " Analiza cómo estos sesgos afectan la forma en que se perciben las mujeres en contextos profesionales y sociales."

    # ----------------------
    # 3. Agente Redactor
    # ----------------------
    elif tipo == "redactor":
        goal = "Reescribir el texto original usando lenguaje inclusivo y eliminando estereotipos."
        backstory = contexto + " Se especializa en transformar lenguaje sesgado en narrativas igualitarias."

    # ----------------------
    # 4. Agente Comparador
    # ----------------------
    elif tipo == "comparador":
        goal = "Comparar los análisis de otros agentes y reflexionar sobre cómo la identidad influye en la detección de sesgos."
        backstory = contexto + " Observa cómo las variables de identidad (profesión, género, política) afectan la interpretación del texto."
    else:
        raise ValueError(f"Tipo de agente no válido: {tipo}")

    return Agent(
        role=role,
        goal=goal,
        backstory=backstory,
        verbose=True,
        llm=llm
    )

In [ ]:
# ----------------------
# 5. Agente Estilizador
# ----------------------
def crear_estilizador():
    return Agent(
        role="Estilizador (asistente técnico neutral)",
        goal="Convertir los resultados anteriores en un informe estructurado en Markdown.",
        backstory="Asistente especializado en presentación clara de resultados. No realiza análisis, sino que organiza y mejora la visualización del contenido.",
        verbose=True,
        llm=llm
    )

## Crear agentes

In [ ]:
# Define los parámetros para cada uno de los 5 agentes funcionales
profesion = "juez" # "jueza", "abogada", "periodista", "investigadora", "profesora", "médica", "psicóloga", "trabajadora social"
genero = "mujer" # "mujer", "hombre"
politica = "progresista" # "progresista", "conservadora", "neutra"

# Define los perfiles de los agentes
perfiles = {
    "analista":    {"profesion": profesion, "genero": genero, "politica": politica},
    "critico":     {"profesion": profesion, "genero": genero, "politica": politica},
    "redactor":    {"profesion": profesion, "genero": genero, "politica": politica},
    "comparador":  {"profesion": profesion, "genero": genero, "politica": politica},
}

In [ ]:
# Crear agentes
agentes = {
    tipo: crear_agente(tipo, **params)
    for tipo, params in perfiles.items() if tipo != "estilizador"
}
agentes["estilizador"] = crear_estilizador()


## Crear tareas

In [ ]:
tareas = [
    Task(
    description=(
            f"""Analiza el siguiente texto completo con atención al contexto y la intención del autor.

    Tu objetivo es detectar frases que perpetúen estereotipos, lenguaje sexista o sesgos de género de forma explícita o implícita.  
    ⚠️ Ten en cuenta que algunas frases podrían parecer problemáticas en sí mismas, pero no lo son si el texto las está usando para **criticar** esos sesgos.  
    No marques como sesgada ninguna frase si forma parte de una **denuncia o crítica** a ese mismo lenguaje.

    1. Enumera las frases que verdaderamente refuercen o reproduzcan estereotipos o sesgos de género. Justifica cada una.
    2. En el texto original:
        - Usa `<span style="color: red; font-weight: bold;">... </span>` para marcar frases con sesgos o estereotipos.

    Texto original:
    {texto_noticia}
    """
        ),
        expected_output=(
            "Frases problemáticas con justificación + texto original anotado con Markdown."
        ),
        agent=agentes["analista"]
    ),
    Task(
        description="Explica cómo los sesgos detectados afectan la percepción de la mujer en la sociedad.",
        expected_output="Análisis crítico del impacto social.",
        agent=agentes["critico"],
        input="previous_task_output"
    ),
    Task(
        description="Reescribe el texto con lenguaje inclusivo y sin estereotipos.",
        expected_output="Nueva versión del texto con lenguaje igualitario.",
        agent=agentes["redactor"],
        input="previous_task_output"
    ),
    Task(
        description="Compara los análisis anteriores y reflexiona sobre cómo influyen la profesión, género y política de cada agente.",
        expected_output="Reflexión comparativa desde una perspectiva neutral.",
        agent=agentes["comparador"],
        input="previous_task_output"
    ),
    Task(
        description="Toma todos los análisis anteriores y preséntalos en un informe en formato Markdown con títulos, listas, separadores y claridad visual.",
        expected_output="Informe en Markdown limpio y bien estructurado.",
        agent=agentes["estilizador"],
        input="previous_task_output"
    )
]

# Orchestrate Agents

In [ ]:
crew = Crew(
    agents=list(agentes.values()),
    tasks=tareas,
    verbose=True
)

# Execute

In [ ]:
resultado = crew.kickoff()

In [ ]:
analista_output = resultado.tasks_output[0].raw
display(Markdown(analista_output))

In [ ]:
critico_output = resultado.tasks_output[1].raw
display(Markdown(critico_output))

In [ ]:
redactor_output = resultado.tasks_output[2].raw
display(Markdown(redactor_output))

In [ ]:
comparador_output = resultado.tasks_output[3].raw
display(Markdown(comparador_output))

In [ ]:
# display(Markdown(resultado.raw))